In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

#Cargamos los datos a trabajar

x_train_pre=np.load("./data/train_signals.npy")
y_train_pre=np.load("./data/train_marks.npy")
trainLen=len(x_train_pre)

x_val_pre=np.load("./data/val_signals.npy")
y_val_pre=np.load("./data/val_marks.npy")
valLen=len(x_val_pre)

x_test_pre=np.load("./data/test_signals.npy")
y_test_pre=np.load("./data/test_marks.npy")
testLen=len(x_test_pre)

#Creamos una funcion que muestree a un ratio deseado

def sampling(signals,samples,lenght,rate):
    x_new=[]
    for i in signals:
        x=[]
        for j in range(0,lenght,rate):
            x.append(i[j])
        x_new.append(x)
    return np.array(x_new)

#Obtenemos los nuevos resultados de nuestras señales a trabajar

x_train=sampling(x_train_pre,trainLen,800,4)
x_val=sampling(x_val_pre,valLen,800,4)
x_test=sampling(x_test_pre,testLen,800,4)

#Creamos una funcion que clasifique con 1 o 0 los valores de la señal a trabajar

def oneHot(marks,lenght,rate):
    y_new=[]
    for i in marks:
        y=[]
        for j in range(0,lenght,rate):
            if(i[0]<=j & j<=i[1]):
                y.append(1)
            else:
                y.append(0)
        y_new.append(y)
    return np.array(y_new)

#Obtenemos los nuevos resultados de nuestras marcas a trabajar

y_train=oneHot(y_train_pre,800,4)
y_val=oneHot(y_val_pre,800,4)
y_test=oneHot(y_test_pre,800,4)






In [2]:
model = tf.keras.Sequential()

model.add(tf.keras.layers.Conv1D(40, 7, activation='elu',input_shape=(None,200)))
model.add(tf.keras.layers.MaxPool1D(pool_size=5))

model.add(tf.keras.layers.Conv1D(40, 7, activation='elu'))
model.add(tf.keras.layers.MaxPool1D(pool_size=5))

model.add(tf.keras.layers.Conv1D(40, 7, activation='elu'))
model.add(tf.keras.layers.MaxPool1D(pool_size=5))

model.add(tf.keras.layers.Conv1D(40, 7, activation='elu'))
model.add(tf.keras.layers.MaxPool1D(pool_size=5))

model.add(tf.keras.layers.Conv1D(40, 7, activation='elu'))
model.add(tf.keras.layers.MaxPool1D(pool_size=5))

model.build(input_shape=(None,200,1))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, None, 40)          56040     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, None, 40)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, None, 40)          11240     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, None, 40)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, None, 40)          11240     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, None, 40)          0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, None, 40)          1